In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.scrapethissite.com/pages/forms/"
all_rows = []
columns = []

# Scrape alle Seiten
page = 1
while True:
    resp = requests.get(BASE_URL, params={"page_num": page})
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    if not table:
        break

    if not columns:
        columns = [th.get_text(strip=True) for th in table.find_all("th")]

    rows = table.select("tr.team")
    if not rows:
        break

    for row in rows:
        cells = [td.get_text(strip=True) for td in row.find_all("td")]
        all_rows.append(cells)

    page += 1

# DataFrame erstellen
df = pd.DataFrame(all_rows, columns=columns)

# Numerische Spalten konvertieren
for col in ["Year", "Wins"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# CSV speichern
df.to_csv("hockey_teams.csv", index=False)

# Fragen beantworten
for year in [1990, 2000, 2010]:
    subset = df[df["Year"] == year]
    top_team = subset.loc[subset["Wins"].idxmax(), "Team Name"]
    print(f"{year}: {top_team} mit {subset['Wins'].max()} Wins")

for year in [1991, 2001, 2011]:
    print(f"{year}: {df[df['Year'] == year]['Team Name'].nunique()} Teams")


1990: Chicago Blackhawks mit 49 Wins
2000: Colorado Avalanche mit 52 Wins
2010: Vancouver Canucks mit 54 Wins
1991: 22 Teams
2001: 30 Teams
2011: 30 Teams
